In [88]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import FastText
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, f1_score

In [ ]:
'''
 데이터 확인하기
 불용어 처리, 특수 문자 제거 등 전처리 포함하여 tokenizing하기
 One-hot encoding, Word2Vec, CBOW, Skip-gram, GloVe 등의 방법으로 임베딩하기
 유사도, Wordcloud, 이진 분류, 그래프 해석 등 유의미한 해석 도출하기
 토크나이저, 임베딩 모델 선택 과정, 인사이트 해석 주석으로 달기
'''

In [60]:
df = pd.read_csv("../spam.csv")
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [7]:
df.isnull().sum()

v1    0
v2    0
dtype: int64

In [9]:
df['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [46]:
df[df.duplicated()]

,v1,v2
102,ham,As per your request 'Melle Melle (Oru Minnamin...
153,ham,As per your request 'Melle Melle (Oru Minnamin...
206,ham,"As I entered my cabin my PA said, '' Happy B'd..."
222,ham,"Sorry, I'll call later"
325,ham,No calls..messages..missed calls
...,...,...
5524,spam,You are awarded a SiPix Digital Camera! call 0...
5535,ham,"I know you are thinkin malaria. But relax, chi..."
5539,ham,Just sleeping..and surfing
5553,ham,Hahaha..use your brain dear


5500여개의 데이터 중 약 400여개의 중복 데이터가 확인되었으며 모두 제거한다.

In [61]:
# 중복 제거
df.drop_duplicates(subset='v2',keep='first', inplace = True)

In [78]:
df.replace({'v1': {'spam': 1, 'ham': 0}}, inplace = True)

## 정제

In [62]:
def cleaning(text) :
    text = text.lower() 
    
    pattern = '[^a-zA-Z]' # 특수 문자 제거
    text = re.sub(pattern = pattern, repl = ' ', string = text)
    
    pattern = r'\s+'      # 연속 공백 제거
    text = re.sub(pattern = pattern, repl = ' ', string = text)
    
    return text

In [63]:
df['v2'] = df['v2'].apply(cleaning)

## Tokenization

In [64]:
#nltk.download('punkt')
#nltk.download('stopwords')

# 불용어 리스트
stop_word_list = stopwords.words('english')
stop_word_list[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [65]:
# 토큰화 및 불용어 제거
def remove_stopwords(text):
    tokens = word_tokenize(text)
    tokens_wo_stop = [word for word in tokens if word.lower() not in stop_word_list]
    return tokens_wo_stop

df['v2_tokenized'] = df['v2'].apply(remove_stopwords)
df.head()

,v1,v2,v2_tokenized
0,ham,go until jurong point crazy available only in ...,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,ok lar joking wif u oni,"[ok, lar, joking, wif, u, oni]"
2,spam,free entry in a wkly comp to win fa cup final ...,"[free, entry, wkly, comp, win, fa, cup, final,..."
3,ham,u dun say so early hor u c already then say,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,nah i don t think he goes to usf he lives arou...,"[nah, think, goes, usf, lives, around, though]"


## Word Embedding

### 1. CBOW

단어의 문맥적 의미를 보존하면서 단어를 벡터로 표현하는 방법으로, 주변 N개의 단어들로 target 단어를 예측한다.

In [107]:
cbow = Word2Vec(sentences=df['v2_tokenized'].tolist(), size=100, window=5, min_count=5, sg=0) # sg=0

### 2. Skip-gram
단어의 문맥적 의미를 보존하면서 단어를 벡터로 표현하는 방법으로, target 단어로 주변 N개 단어 등장여부를 예측한다. 

In [69]:
skip_gram = Word2Vec(sentences=df['v2_tokenized'].tolist(), size=100, window=5, min_count=5, sg=1) # sg=1

### 3. FastText

In [75]:
fast_text = FastText(df['v2_tokenized'].tolist(), size=100, window=5, min_count=5, workers=5, sg=1)

## 이진분류 SVM

In [104]:
import numpy as np

def SentenceVector(text, size, model):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in text:
        try:
            vec += model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:  # 모델에 없는 단어는 무시
            continue
    if count != 0:
        vec /= count
    return vec

In [112]:
def svc_model(embedding_model) :
    sent_vec = np.concatenate(
        [SentenceVector(z, 100, embedding_model) for z in df['v2_tokenized']])
    
    X_train, X_test, y_train, y_test = train_test_split(sent_vec, df['v1'], test_size=0.2, random_state=42)
    
    svm = SVC(kernel='linear')
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)
    
    return classification_report(y_test, y_pred)

In [115]:
# cbow 
print(svc_model(cbow))

              precision    recall  f1-score   support

           0       0.86      1.00      0.92       889
           1       0.00      0.00      0.00       145

    accuracy                           0.86      1034
   macro avg       0.43      0.50      0.46      1034
weighted avg       0.74      0.86      0.79      1034



In [116]:
print(svc_model(skip_gram))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       889
           1       0.86      0.83      0.85       145

    accuracy                           0.96      1034
   macro avg       0.92      0.90      0.91      1034
weighted avg       0.96      0.96      0.96      1034



In [117]:
print(svc_model(fast_text))

              precision    recall  f1-score   support

           0       0.97      0.98      0.98       889
           1       0.88      0.83      0.86       145

    accuracy                           0.96      1034
   macro avg       0.93      0.91      0.92      1034
weighted avg       0.96      0.96      0.96      1034



In [119]:
sent_vec = np.concatenate(
        [SentenceVector(z, 100, cbow) for z in df['v2_tokenized']])
    
X_train, X_test, y_train, y_test = train_test_split(sent_vec, df['v1'], test_size=0.2, random_state=42)
    
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)
    
# 결정 함수를 사용하여 테스트 샘플에 대한 점수 계산
decision_scores = svm.decision_function(X_test)

# 임계값의 범위 설정 (예: -2에서 2까지 0.01 단위로)
thresholds = np.arange(-2, 2, 0.01)

# 각 임계값에 대한 F1-Score 계산
f1_scores = []
for thresh in thresholds:
    # 임계값을 적용하여 이진 예측 생성
    y_pred_thresh = (decision_scores > thresh).astype(int)
    
    # F1-Score 계산 및 저장
    f1 = f1_score(y_test, y_pred_thresh)
    f1_scores.append(f1)

# 최대 F1-Score 및 해당 임계값 찾기
max_f1 = np.max(f1_scores)
optimal_thresh = thresholds[f1_scores.index(max_f1)]

print(f'최적의 임계값: {optimal_thresh}, 최대 F1-Score: {max_f1}')

최적의 임계값: -0.9899999999999991, 최대 F1-Score: 0.785498489425982


- 스팸 메일 데이터는 'ham'으로 편향된 데이터이며, 스팸 메일을 더 정밀하게 분류하기 위해 최종 평가 지표는 f1 score 값으로 설정하되, recall에 집중하고자 한다. 

- skip_gram과 fast_text의 recall 값은 동일하며 모두 높은 f1 score 값을 보이고 있다.반면, cbow 임베딩 모델을 적용한 경우는 recall 및 f1-score 값이 0으로 나왔다. 이는 데이터 불균형 문제 또는 임계값 설정에 따른 결과일 수 있을 것이라 판단하여, 위와 같이 최적의 임계값을 찾아나갔다. 그러나, 이에 대한 최적의 f1-score 값도 약 0.78로 다른 결과보다 현저히 낮은 것으로 나타났다.
- 따라서, f1-score 기준 fast_text의 성능이 가장 좋은 것으로 나타났다. 
